In [1]:
#Mount to Drive
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [2]:
#Set path
path = '/content/drive/Shareddrives/ECEN_STAT_Midterm/'

In [3]:
#Imports
import pandas as pd
import matplotlib.pyplot as plt

# Loading and Preprocessing the Data

In [ ]:
#Load data
df = pd.read_csv(path+"properties_2016.csv")
df.head()

In [ ]:
#Merge the log error and other associated data
log_df = pd.read_csv(path+"train_2016_v2.csv")
df = df.merge(log_df, on='parcelid', how = 'left')
df.head()

In [ ]:
#Checking the Null situation
print("----\n Vars missing their data \n----\n")
n_null = df.isnull().sum() / len(df) * 100
print(n_null)

In [ ]:
#Check the data for abnormality
print("----\n Checking Data Abnormality \n----\n")
df.describe()

In [ ]:
#Check the data types
print("----\n Checking Data Types \n----\n")
print(df.dtypes)

TO DO:


*   Fix the var types of the following types

  *  parcelid -> object
      * Nothing else comes to mind for now but that is subject to change
*   Drop due too many Null values, where the null values can't provide insight
    
    * buildingclasstypeid
    * decktypeid
    * finishedsquarefeet6
    * finishedsquarefeet13
    * finishedsquarefeet50
    * storytypeid
    * yardbuildingsqft26
    * yardbuildingsqft17



In [ ]:
# Fix variable types
df['parcelid'] = df['parcelid'].astype(str)

# Drop columns with many null values
df.drop(['buildingclasstypeid', 'decktypeid', 'finishedsquarefeet6', 'finishedsquarefeet13',
         'finishedsquarefeet50', 'storytypeid', 'yardbuildingsqft26', 'yardbuildingsqft17'], axis=1, inplace=True)

#Salim: Doing Analysis Based on the Location Data

In [ ]:
df[['longitude', 'latitude']].mean()

In [ ]:
# Create a scatter plot of longitude vs. latitude
df.plot(kind='scatter', x='longitude', y='latitude', figsize=(10, 10))

# Show the plot
plt.show()

Judging by the map this is Los Angeles

CHECK ME:
  * Verify that the data is indeed limited to LA by checking
    1. Zip Codes
    2. Region County Id

Verify the ZipCodes

A zipcode is from California if its in the range [90001, 96161] per https://www.zipcodestogo.com/California/

In [ ]:
#See if the zip code is in California
is_in_california = df['regionidzip'].isin(range(90001, 96162))
counts = is_in_california.value_counts()
counts.plot(kind='bar')
plt.xlabel('is_in_california')
plt.ylabel('Count')
plt.show()

CHECK ME
  * The zip codes are very off. This is due to one of two things. Either the initial assumption is wrong- or the listing zipIds are different bc of the agent/lister

In [ ]:
#Verify Counties
county_counts = df['regionidcounty'].value_counts()
county_counts.plot(kind='bar', figsize=(10, 6))

In [ ]:
#Ok so we have 3 counties in the dataset- lets make sure they overlay logically
plt.scatter(df['longitude'], df['latitude'], c=df['regionidcounty'])
plt.legend()
# Show the plot
plt.show()

In [ ]:
#Finding what each county is quickly
county_names = {3101: 'Los Angeles', 1286: 'Orange', 2061: 'Ventura'}
for county_code, county_name in county_names.items():
    temp_df = df[df['regionidcounty'] == county_code]
    plt.scatter(temp_df['longitude'], temp_df['latitude'], label=county_name)
plt.legend()
plt.show()

In [ ]:
#Now that the changes are verified rename the counties
county_names = {3101: 'Los Angeles', 1286: 'Orange', 2061: 'Ventura'}
df['regionidcounty'] = df['regionidcounty'].replace(county_names)

Now we have verified that the map is indeed in LA since there are 3 counties in the dataset all belonging to LA.

Relationship of Location & Prediction Strength (as evaluated by Log Loss)

In [ ]:
#Can we quantify the predictions based on location perhaps?
subset = df[df['logerror'].notnull()]
plt.hexbin(subset['longitude'], subset['latitude'], C=subset['logerror'], gridsize=10000, cmap='YlOrRd')
plt.colorbar()
plt.show()